In [2]:
# Imports
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes

In [ ]:
# Helper functions

def decodeBase64(s):
    base64_chars = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
    char_to_val = {ch: i for i, ch in enumerate(base64_chars)}
    s = s.rstrip('=')
    bits = ""
    for char in s:
        if char not in char_to_val:
            raise ValueError(f"Invalid base64 character: {char}")
        val = char_to_val[char]
        bits += f"{val:06b}"
    bytes_out = bytearray()
    for i in range(0, len(bits), 8):
        byte = bits[i:i+8]
        if len(byte) == 8:
            bytes_out.append(int(byte, 2))

    return bytes(bytes_out)

## keyToeN(string) -> tuple
## Takes in the path to a file containing a
## public RSA key, and returns a tuple with e
## and N values.
## Intended for use with rsaEncrypt(), to be
## piped into the key parameter in tuple form,
## or users can use the string form and just
## pass the file path to the function. 
def parsePublicKey(file: str | None = None,
               key: str | int | None = None):
    if (file == None) and (key == None):
        raise ValueError("Either file or key must be provided")
    if (file != None) and (key != None):
        raise ValueError("Only one of file or key can be provided")
    if file != None:
        with open(file, "r") as keyfile:
            keyDER = keyfile.read().split("\n")[1]
    if key != None:
        keyDER = key.split("\n")[1]
    der = decodeBase64(keyDER)
    modulus_offset = der.find(b'\x02\x82\x01\x01') + 4
    exponent_offset = modulus_offset + 257
    assert der[exponent_offset] == 0x02
    e_len = der[exponent_offset + 1]
    modulus = der[modulus_offset:modulus_offset + 257]
    publicExponent = der[exponent_offset + 2:exponent_offset + 2 + e_len]

    return (int(publicExponent.hex(), 16), int(modulus.hex(), 16))

print(parsePublicKey("Practice Keys/practicepublic.pem"))

def parsePrivateKey(file: str | None = None,
                key: str | int | None = None,
                publicExp: bool = False):
    if (file == None) and (key == None):
        raise ValueError("Either file or key must be provided")
    if (file != None) and (key != None):
        raise ValueError("Only one of file or key can be provided")
    if file != None:
        with open(file, "r") as keyfile:
            keyDER = keyfile.read().split("\n")[1]
    if key != None:
        keyDER = key.split("\n")[1]
    der = decodeBase64(keyDER)

    def read_length(data, offset):
        first = data[offset]
        offset += 1
        if first & 0x80 == 0:
            return first, offset
        num_bytes = first & 0x7F
        length = int.from_bytes(data[offset:offset+num_bytes], "big")

        return length, offset + num_bytes

    def read_integer(data, offset):
        assert data[offset] == 0x02
        length, offset = read_length(data, offset + 1)
        value = int.from_bytes(data[offset:offset+length], "big")

        return value, offset + length

    def read_sequence(data, offset):
        assert data[offset] == 0x30
        length, offset = read_length(data, offset + 1)

        return offset, offset + length

    offset, _ = read_sequence(der, 0)
    _, offset = read_integer(der, offset)
    alg_start, alg_end = read_sequence(der, offset)
    offset = alg_end
    assert der[offset] == 0x04
    pk_len, offset = read_length(der, offset + 1)
    rsa_data = der[offset:offset + pk_len]
    offset, _ = read_sequence(rsa_data, 0)
    version, offset = read_integer(rsa_data, offset)
    modulus, offset = read_integer(rsa_data, offset)
    publicExponent, offset = read_integer(rsa_data, offset)
    privateExponent, _ = read_integer(rsa_data, offset)

    if publicExp:
        return (publicExponent, modulus)
    
    if not publicExp:
        return (privateExponent,modulus)


print(parsePrivateKey("Practice Keys/practiceprivate.key"))

def encodetext(file: str | None = None,
               text: str | None = None,
               dest: str | None = None):
    if (file == None) and (text == None):
        raise ValueError("Either file or text must be provided")
    if (file != None) and (text != None):
        raise ValueError("Only one of file or text can be provided")
    if file != None:
        textfile = open(file, "r")
        text = textfile.read()
        encodetext(text=text)
    if text != None:
        if dest != None:
            with open(dest, "w") as file:
                file.write(text.encode("ascii").hex())
        return text.encode("ascii").hex()

def decodeText(file: str | None = None,
               text: str | None = None,
               dest: str | None = None):
    if (file == None) and (text == None):
        raise ValueError("Either file or text must be provided")
    if (file != None) and (text != None):
        raise ValueError("Only one of file or text can be provided")
    if file != None:
        textfile = open(file, "r")
        text = textfile.read()
        decodeText(text=text)
    if text != None:
        if dest != None:
            with open(dest, "w") as file:
                file.write(bytes.fromhex(text).decode("ascii"))
        return bytes.fromhex(text).decode("ascii")

(65537, 25082364543938453821300195307385660648625807832855021105558116089547058422553931018464936723362771231022120173300125644744743375324538912334304910508187990643572360906423320129999572319421737753704847165752025742472634051219053842807942395961764868214518255052296707972421268420530868134630391725646800215849024813713974818165972799108430355161496216949676765347473403156646951426563476078274736116557071076308241395990128772317868877331476079857572643218843444384647553931198093171887944469338676929035658606414465537809003764732191866545083465917363102221981297958029810371399345025874461830676938724223288164066543)
(987285376135172992357249322134022300261341916414002093396905475844180680527659919614615774576541080704124575507784444198937657325723588281477480848870823352666302101290185967825097223458447682932461189726905613723080556483013141364412600542331305099250530488817876966004799198636227687924367761952252440252286793061797806656166217264445148754193413086015068262321875386915

In [ ]:
def rsaEncrypt(plaintext, key: tuple | str | int):
    if type(key) == tuple:
        ciphertext = (plaintext ** key[0]) % key[1]
    if (type(key) == str) or (type(key) == int):
        ciphertext = (plaintext ** parsePublicKey(key)[0]) % parsePublicKey(key)[1]
    if (type(key) != str) and (type(key) != int) and (type(key) != tuple):
        raise TypeError("Invalid key type")
    return ciphertext

def rsaDecrypt(ciphertext, key: tuple | str | int):
    if type(key) == tuple:
        plaintext = (ciphertext ** key[0]) % key[1]
    if (type(key) == str) or (type(key) == int):
        plaintext = (ciphertext ** parsePrivateKey(key)[0]) % parsePrivateKey(key)[1]
    if (type(key) != str) and (type(key) != int) and (type(key) != tuple):
        raise TypeError("Invalid key type")
    return ciphertext


In [ ]:
data = open("data.txt", "r").read()
encodedData = data.encode("ascii").hex()
with open("encodeddata.txt", "w") as file:
    file.write(encodedData)

In [ ]:
decodedData = bytes.fromhex(encodedData).decode("ascii")
with open("decodeddata.txt", "w") as file:
    file.write(decodedData)

In [ ]:
data == open("decodeddata.txt", "r").read()

True

In [62]:
def rsaBruteForce():
    ...